In [13]:
import pickle
import time
import numpy as np
import random
import graphviz as gv
import imageio
import os
import imageio.v2 as imageio

class States():
    def __init__(self, state,transitions_with_action, transitions_without_action, resolution = False):
        """
        Initialisation des états de la chaine de markov
        
        Args
        ----
        state : str
            Nom de l'état (S0...)
        reward : int
            Récompense de l'état
        transitions_with_action : dict
            Dictionnaire de l'ensemble des transitions avec action
        transitions_without_action : dict
            Dictionnaire de l'ensemble des transitions sans action
        resolution : bool
            Booléen qui permet de savoir si on veut résoudre les problèmes détéctés automatiquement si possible
        """
        self.state = state
        self.reward = 0 # récompense de l'état
        self.transitions_without_action = {}
        self.transitions_with_action = {}

        sans_action = True # on crée ce booléen pour vérifier qu'il n'y a pas de transition avec action ensuite
        for transition in transitions_without_action:
            # on parcours l'ensemble des transitions
            transi_active = transitions_without_action[transition]
            if transi_active["from"]==self.state: # on vérifie si l'état de départ est l'état actif
                sans_action = False
                self.transitions_without_action["targets"] = transi_active["targets"]
                self.transitions_without_action["weights"] = transi_active["weights"]

        for transition in transitions_with_action:
            transi_active = transitions_with_action[transition]
            if transi_active["from"]==self.state and sans_action == True:
                # les clés dans transitions_with_action sont les actions et les valeurs des dict des états cibles et leurs poids
                self.transitions_with_action[transi_active["action"]] = {"targets" : transi_active["targets"], "weights" : transi_active["weights"]}
            elif transi_active["from"]==self.state and sans_action == False:
                print("\nWarning : l'état", self.state, "comporte des transitions avec et sans action")
                if resolution == True: # On choisit pour résoudre le problème de supprimer les transitions avec action de l'état, pour ça on ne rajoute pas la transition dans self.transitions_with_action
                    print("Le problème a été résolu automatiquement : une transition avec action de l'état", self.state, "a été supprimée")
                    pass

    def __repr__(self):
        return (f"State : {self.state} \n State reward {self.reward} \n Transitions without action : {self.transitions_without_action} \n Transitions with action : {self.transitions_with_action}")


class markov():
    def __init__(self, fichier_mdp):
        """
        Initialisation de la chaine de markov

        Args
        ----
        fichier_mdp : str
            Nom du fichier contenant la modélisation de la chaine de markov
        """
        ! python mdp_lecture/mdp.py < {fichier_mdp} # On lance le fichier mdp.py avec la modélisation voulue
        # On lit le .pickle des données que l'on souhaitent récupérer
        def read_list(nom_liste):
            # for reading also binary mode is important
            with open(nom_liste, 'rb') as fp:
                liste = pickle.load(fp)
                return liste

        L = read_list("liste_donnees")
        # print('Données récupérées', L)
        states, actions, transitions_with_action, transitions_without_action = L['States'], L['Actions'], L['Transitions_with_action'], L['Transitions_without_action']
        print("-"*50 + "\n")

        # Pour savoir si on veut résoudre les problèmes détectés automatiquement

        print("\n Souhaitez-vous que les problèmes détéctés prochainement soient résolus automatiquement si cela est possible? (y/n)")
        time.sleep(0.5)
        reponse_resolution_pb = input()
        if reponse_resolution_pb == "y":
            resolution = True
        else:
            resolution = False


        # on peut alors initialiser les éléments de la chaine de markov
        self.states = {}
        self.liste_states = states
        self.actions = actions
        self.transitions_with_action = transitions_with_action
        self.transitions_without_action = transitions_without_action

        for state in states:
            self.states[f"{state}"] = States(state, transitions_with_action, transitions_without_action, resolution)
        
        # On vérifie que la chaine de markov est correctement définie et on résout les problèmes si possibles et si souhaité
        markov.parsing(self, states, resolution) # on vérifie qu'on a pas de problèmes de parsing

        print("Souhaitez-vous ajouter des récompenses aux états ? (y/n)")
        time.sleep(0.5)
        reponse = input()
        if reponse == "y":
            for state in self.states :
                etat = self.states[state]
                reward = markov.affichage_etat(self, etat, reward = True)
                etat.reward = float(reward)
        
        print("Que souhaitez-vous faire ?")
        print("1. lancer un parcours")
        print("2. model checking d'un eventually")
        print("3. SMC quantitatif")
        time.sleep(0.5)
        reponse = input()
        while reponse not in ["1", "2", "3"]:
            reponse = input()
        if reponse == "1":
            markov.parcours(self)
        elif reponse == "2":
            print("Quel sont les états cibles ? Veuillez les rentrer un par un, et taper 'fin' quand vous avez fini")
            time.sleep(0.5)
            etat = input()
            etats_cibles = []
            while etat != "fin":
                if etat not in self.liste_states:
                    print("L'état", etat, "n'existe pas")
                    etat = input()
                else :
                    etats_cibles.append(etat)
                    etat = input()
            markov.eventually_check_DTMC(self, etats_cibles)
        elif reponse == "3" :
            if self.transitions_with_action == {}:
                markov.smc_quantitatif(self)
            else :
                print("SMC quantitatif impossible car la chaine de markov n'est pas déterministe")
                return(False) # changer pour pouvoir lancer autre chose


        
    def parsing(self, states, resolution):
        """
        On réalise différents tests pour vérifier que la chaine de markov (ou la MDP) est correctement définie

        Args
        ----
        states : list
            Liste des états de la chaines (avec possiblement des doublons)
        resolution : bool
            Booléen qui permet de savoir si on veut résoudre les problèmes détéctés automatiquement si possible
        """
        # On concatène les transitions avec et sans action pour plus de simplicité pour les test
        Warning = False # passe à True si on a un warning

        all_transitions = []
        transitions_without_action = []
        transitions_with_action = []
        for elem in self.transitions_without_action:
            transitions_without_action.append(self.transitions_without_action[elem])
            all_transitions.append(self.transitions_without_action[elem])
        for elem in self.transitions_with_action:
            transitions_with_action.append(self.transitions_with_action[elem])
            all_transitions.append(self.transitions_with_action[elem])

        # On vérifie que les états déclarés sont utilisés et qu'un état utilisé dans des transitions est déclaré
        used_states = []
        for transitions in all_transitions:
            if transitions["from"] not in self.states:
                print(f"Warning : l'état {transitions['from']} est utilisé dans une transition mais n'est pas déclaré")
                Warning = True
                if resolution:
                    # On rajoute l'état manquant
                    self.states[transitions["from"]] = States(transitions["from"], self.transitions_with_action, self.transitions_without_action)

            used_states.append(transitions["from"])
            used_states.append(transitions["targets"])
            for target in transitions["targets"]:
                if target not in self.states:
                    print(f"Warning : l'état {target} est utilisé dans une transition mais n'est pas déclaré")
                    Warning = True
                    if resolution:
                        # On rajoute l'état manquant
                        print("Rajout de l'état manquant")
                        self.states[target] = States(target, self.transitions_with_action, self.transitions_without_action)
        for state in self.states:
            if state not in used_states:
                print(f"Warning : l'état {state} est déclaré mais n'est pas utilisé")
                Warning = True
                if resolution:
                    # On rajoute une transition de l'état vers lui même pour éviter les erreurs
                    print("Rajout d'une transition de l'état vers lui même")
                    self.transitions_without_action[f"{state}"] = {'from': f'{state}', 'targets': [f"{state}"], 'weights': [10]}
                    self.states[f"{state}"].transitions_without_action = self.transitions_without_action[f"{state}"]

        # On vérifie qu'un état n'est pas déclaré plusieurs fois
        if len(states) != len(self.states): # self.states supprime automatiquement les doublons
            print("Warning : un état est déclaré plusieurs fois")
            Warning = True

        # On vérifie qu'un état possède bien une transition de sortie
        etats_sans_sortie = []
        dict_etats = self.states
        bool_etat_sans_sortie = False
        for state in dict_etats:
            if dict_etats[state].transitions_without_action == {} and dict_etats[state].transitions_with_action == {}:
                etats_sans_sortie.append(state)
                bool_etat_sans_sortie = True
                if resolution:
                    # On rajoute une transition de l'état vers lui même pour pour lui créer un état de sortie
                    self.transitions_without_action[f"{state}"] = {'from': f'{state}', 'targets': [f"{state}"], 'weights': [10]}
                    self.states[f"{state}"].transitions_without_action = {'from': f'{state}', 'targets': [f"{state}"], 'weights': [10]}

        if bool_etat_sans_sortie == True:
            print("Warning : les états suivants n'ont pas de transition de sortie : ", etats_sans_sortie)
            print(f"Rajout d'une transition des états {etats_sans_sortie} vers eux mêmes")

        # On vérifie que les actions ne sont pas déclarées plusieurs fois
        actions_uniques = set(self.actions)
        if len(actions_uniques) != len(self.actions):
            print("Warning : une action est déclarée plusieurs fois")
            Warning = True
            if resolution:
                print("Suppressions automatiques des actions déclarées plusieurs fois")
    
        # On vérifie qu'une action déclarée est utilisée
        actions_utilisés = []
        for transitions in transitions_with_action:
            actions_utilisés.append(transitions["action"])
        actions_utilisés = set(actions_utilisés) # on enlève les doublons
        if len(actions_utilisés) != len(self.actions) and self.actions != ["None"]: # On met None de base s'il n'y a pas d'action
            print("Warning : une action est déclarée mais n'est pas utilisée")
            Warning = True
            if resolution:
                print("Suppression des actions non utilisées")
                # On supprime les actions non utilisées
                for action in self.actions:
                    if action not in actions_utilisés:
                        self.actions.remove(action)

        # On vérifie que les actions utilisées sont déclarées
        for action in actions_utilisés:
            if action not in self.actions:
                print(f"Warning : l'action {action} est utilisée mais n'est pas déclarée")
                Warning = True
                if resolution:
                    print("Rajout de l'action manquante")
                    # On rajoute l'action manquante
                    if self.actions == ["None"]:
                        self.actions = [action]
                    else:
                        self.actions.append(action)

        if Warning :
            print("\n Écrire ok pour contiuer")
            print("-"*25 + "\n")
            while True:
                rep = input()
                if rep == "ok":
                    break
            
    def affichage_etat(self,etat_actif,choix = False, reward = False):
        """
        Affiche proprement l'état actif de la chaine de markov et permet de choisir une action le cas échéant

        Args
        ----
        etat_actif : état de la chaine de markov
            Etat actif de la chaine de markov
        choix : bool
            Si on doit afficher des choix pour des actions
        """
        nom = etat_actif.state
        print("-"*50)
        chaine_nom = f"| Etat actif : {nom}"
        l = len(chaine_nom)
        print(chaine_nom + " "*(49-l) + "|")
        if choix :
            chaine_choix = f"| choix possibles pour l'état {etat_actif.state}:"
            l = len(chaine_choix)
            print(chaine_choix + " "*(49-l) + "|")

            chaine_transi = f"| {list(etat_actif.transitions_with_action.keys())}"
            l = len(chaine_transi)
            print(chaine_transi + " "*(49-l) + "|")

            chaine_action = f"| choix de l'action  :"
            l = len(chaine_action)
            print(chaine_action + " "*(49-l) + "|")
            time.sleep(0.5)
            choix = input()
            while choix not in list(etat_actif.transitions_with_action.keys()): # on vérifie que l'action choisie est valide
                choix = input()
                if choix == "stop": # permet d'arrêter le programme à la main
                    break
            chaine_choix = f"| Vous avez choisi l'action {choix}"
            l = len(chaine_choix)
            print(chaine_choix + " "*(49-l) + "|")

        elif reward:
            chaine_choix = f"| Récompense pour l'état {etat_actif.state}:"
            l = len(chaine_choix)
            print(chaine_choix + " "*(49-l) + "|")
            time.sleep(0.5)
            choix = input()
            chaine_rec = f"| Vous avez choisi : {choix}"
            l = len(chaine_rec)
            print(chaine_rec + " "*(49-l) + "|")

            
        print("-"*50)
        print("\n")   
        return choix    

    def parcours(self, file_name="visu_parcours"): # on parcours la chaine (en faisant N étapes)
        """ 
        On parcours la chaine en faisant N étapes
        
        Args
        ----
        file_name : str
            Nom du fichier de sauvegarde de la vidéo
        """
        positionnel = False
        n_pos = False
        without_action = False
        
        print("Quel mode de parcours voulez-vous ?")
        print("1 : sans actions")
        print("2 : avec adversaire positionnel")
        print("3 : avec adversaire non positionnel")
        time.sleep(0.5)
        choix_parcours = input()
        while choix_parcours not in ["1","2","3"]:
            choix_parcours = input()
        if choix_parcours == "1":
            without_action = True
        elif choix_parcours == "2":
            positionnel = True
        elif choix_parcours == "3":
            n_pos = True
        
        print("Combien d'étapes voulez-vous faire ?")
        time.sleep(0.5)
        N = int(input())

        etat_initial = self.liste_states[0] #état initial : premier élément
        etat_actif = self.states[etat_initial] # on prend l'objet correspondant à l'état initial
        reward_total = 0
        
        self.afficher(etat = etat_initial, file_name='image0')
        images = []
        images.append(imageio.imread('image0'+'.png'))

        if without_action:
        # On vérifie qu'on ne choisit pas le mode "sans actions" alors qu'il y en a
            for state in self.states:
                if len(self.states[state].transitions_with_action) != 0:
                    print(f"Error : il y a des transitions avec actions dans l'état {state}, il faut choisir le mode avec actions")
                    without_action = False
                    print("Voulez-vous changer de mode ? (y/n, si non, le programme s'arrête)")
                    time.sleep(0.5)
                    choix = input()
                    while choix not in ["y","n"]:
                        choix = input()
                    if choix == "y":
                        print("Quel mode de parcours voulez-vous ?")
                        print("1 : avec adversaire positionnel")
                        print("2 : avec adversaire non positionnel")
                        time.sleep(0.5)
                        choix_parcours = input()
                        while choix_parcours not in ["1","2"]:
                            choix_parcours = input()
                        if choix_parcours == "1":
                            positionnel = True
                        elif choix_parcours == "2":
                            n_pos = True
                    else :
                        return
        
        if without_action:
            for i in range(N):
                markov.affichage_etat(self, etat_actif)
                # print(etat_actif.state) # on affiche l'état en cours
                poids = etat_actif.transitions_without_action["weights"]
                poids_total = np.sum(poids)
                poids = poids/poids_total # on normalise les poids pour qu'ils soient entre 0 et 1
                poids = np.cumsum(poids) # on fait la somme cumulée des poids, afin de pouvoir faire un tirage aléatoire (il ne faut pas que par exemple les deux probas soient de 0.5, il en faut une de 0.5 et l'autre de 1)

                choix = random.random() # tirage aléatoire entre 0 et 1

                for j in range(len(poids)): # on recherche l'état cible
                    if choix <= poids[j]:
                        reward_total += etat_actif.reward
                        etat_actif = self.states[etat_actif.transitions_without_action["targets"][j]]
                        break
                self.afficher(etat = etat_actif.state, file_name='image'+str(i+1))
                images.append(imageio.imread('image'+str(i+1)+'.png'))
            # create gif
            imageio.mimsave(file_name+'.gif', images, fps=3)
            for i in range(N+1):
                os.remove('image'+str(i)+'.png')

        elif positionnel == True or n_pos == True: # adversaire positionnel
            if positionnel :
                print("Choix d'un adversaire positionnel")
                adv_pos = {} # contient pour chaque état, le choix de l'adversaire
                for state in self.states :
                    etat = self.states[state]
                    if etat.transitions_with_action != {} : # si l'état possèdes des transitions avec actions
                        action = markov.affichage_etat(self, etat, choix = True)
                        adv_pos[state] = action

            for i in range(N):
                if etat_actif.transitions_with_action == {} : # si l'état n'a pas de transitions avec actions :
                    markov.affichage_etat(self, etat_actif)
                    print(etat_actif)
                    poids = etat_actif.transitions_without_action["weights"]

                    poids_total = np.sum(poids)
                    poids = poids/poids_total # on normalise les poids pour qu'ils soient entre 0 et 1
                    poids = np.cumsum(poids) # on fait la somme cumulée des poids, afin de pouvoir faire un tirage aléatoire (il ne faut pas que par exemple les deux probas soient de 0.5, il en faut une de 0.5 et l'autre de 1)

                    choix = random.random() # tirage aléatoire entre 0 et 1

                    for j in range(len(poids)): # on recherche l'état cible
                        if choix <= poids[j]:
                            reward_total += etat_actif.reward
                            etat_actif = self.states[etat_actif.transitions_without_action["targets"][j]]
                            break
                else :
                    if n_pos == True :
                        action_choisie = markov.affichage_etat(self, etat_actif, choix =True)
                    else :
                        action_choisie = adv_pos[etat_actif.state]
                        print(f"Action choisie : {action_choisie}")
                        markov.affichage_etat(self, etat_actif)
                    poids = etat_actif.transitions_with_action[action_choisie]["weights"] # on ne prend que les poids de l'action choisie par l'adversaire
                    poids_total = np.sum(poids)
                    poids = poids/poids_total # on normalise les poids pour qu'ils soient entre 0 et 1
                    poids = np.cumsum(poids) # on fait la somme cumulée des poids, afin de pouvoir faire un tirage aléatoire (il ne faut pas que par exemple les deux probas soient de 0.5, il en faut une de 0.5 et l'autre de 1)

                    choix = random.random() # tirage aléatoire entre 0 et 1

                    for j in range(len(poids)): # on recherche l'état cible
                        if choix <= poids[j]:
                            reward_total += etat_actif.reward
                            etat_actif = self.states[etat_actif.transitions_with_action[action_choisie]["targets"][j]]
                            break
                        
                self.afficher(etat = etat_actif.state, file_name='image'+str(i+1))
                images.append(imageio.imread('image'+str(i+1)+'.png'))
            # create gif
            imageio.mimsave(file_name+'.gif', images, fps=3)
            for i in range(N+1):
                os.remove('image'+str(i)+'.png')
            for i in range(N+1):
                os.remove('image'+str(i))
                
        print(f"Reward total : {reward_total} (si aucun reward n'a été donné, le reward total est de 0)")
        print("Le graphique est visible dans le fichier visu_parcours.gif")

    def parcours_SMC(self, length, etat): # on parcours la chaine (en faisant N étapes)
        """ 
        On parcours la chaine sans les actions, sans afficher le graphe. 
        
        Args:
            length (int): nombre max d'itération 
            etat (str): état d'arrivée
        
        """

        etat_initial = self.liste_states[0] #état initial : premier élément
        etat_actif = self.states[etat_initial] # on prend l'objet correspondant à l'état initial
        reward_total = 0
        if etat_actif.state == etat:
            return etat
                        
        for i in range(length):
            # print(etat_actif.state) # on affiche l'état en cours
            poids = etat_actif.transitions_without_action["weights"]
            poids_total = np.sum(poids)
            poids = poids/poids_total # on normalise les poids pour qu'ils soient entre 0 et 1
            poids = np.cumsum(poids) # on fait la somme cumulée des poids, afin de pouvoir faire un tirage aléatoire (il ne faut pas que par exemple les deux probas soient de 0.5, il en faut une de 0.5 et l'autre de 1)

            choix = random.random() # tirage aléatoire entre 0 et 1

            for j in range(len(poids)): # on recherche l'état cible
                if choix <= poids[j]:
                    reward_total += etat_actif.reward
                    etat_actif = self.states[etat_actif.transitions_without_action["targets"][j]]
                    break
            if etat_actif.state == etat:
                break
            
        return etat_actif.state


    
    def afficher(self, etat = "default", file_name = "graph"):
        
        """Fonction qui permet de représenter le graphe de la chaine de Markov avec ou sans action. Dans le mode "default", le graphe de base est affiché.
        Dans le mode "etat", on peut choisir un état et il sera mis en évidence en étant de couleur bleue.
         """
    
        # On récupère les données
        States = self.states
        Actions = self.actions
        Transitions_with_action = self.transitions_with_action
        Transitions_without_action = self.transitions_without_action

        # On crée le graphique
        G = gv.Digraph(format='png')

        if etat == "default":
            for state in States:
                G.node(state)
        else:
            for state in States:
                if state == etat:
                    G.node(state, color = 'blue', style = 'filled')
                else:
                    G.node(state)
        
        #On ajoute les actions
        for actions in Actions:
            G.node(actions, shape = 'point')
            
        for transition in Transitions_with_action:
            G.edge(Transitions_with_action[transition]['from'], Transitions_with_action[transition]['action'], label=str(Transitions_with_action[transition]['action']), color = 'red')
        

        # On ajoute les transitions sans action
        for transition in Transitions_without_action:
            for i in range(len(Transitions_without_action[transition]['targets'])):
                G.edge(Transitions_without_action[transition]['from'],Transitions_without_action[transition]['targets'][i], label = str(Transitions_without_action[transition]['weights'][i])) 

        # On ajoute les transitions avec action
        for transition in Transitions_with_action:
            for i in range(len(Transitions_with_action[transition]['targets'])):
                G.edge(Transitions_with_action[transition]['action'],Transitions_with_action[transition]['targets'][i], label = str(Transitions_with_action[transition]['weights'][i]))
        
        G.render(file_name, view=False)
    
    def simulation_random(self, n_iter, mode_adv = "random", file_name = "simulation_random" ):
        """ Fonction qui permet de simuler une chaine de Markov avec ou sans action. On peut choisir le nombre d'itération, l'état initial et le mode de l'adversaire.
        Pour l'instant, seul le mode "random" est disponible."""
        
        current_state = self.liste_states[0] #état initial : premier élément
        States = self.states
        Actions = self.actions
        Transitions_with_action = self.transitions_with_action
        Transitions_without_action = self.transitions_without_action
        self.afficher(etat = current_state, file_name='image0')
        reward_total = 0
        images = []
        if mode_adv == "random":
            for i in range(n_iter):
                    actions_possibles = []
                    for transitions in Transitions_with_action:
                        if Transitions_with_action[transitions]['from'] == current_state:
                            actions_possibles.append(Transitions_with_action[transitions]['action'])
                    if len(actions_possibles) == 0:
                        for transitions in Transitions_without_action:
                            if Transitions_without_action[transitions]['from'] == current_state:
                                poids = Transitions_without_action[transitions]['weights']
                                poids_total = np.sum(poids)
                                poids = poids/poids_total
                                poids = np.cumsum(poids)
                                choix = random.random()
                                for j in range(len(poids)):
                                    if choix <= poids[j]:
                                        reward_total += self.states[current_state].reward # ----------
                                        current_state = Transitions_without_action[transitions]['targets'][j]
                                        break
                    else:
                        action_choisie = random.choice(actions_possibles)
                        for transitions in Transitions_with_action:
                            if Transitions_with_action[transitions]['action'] == action_choisie:
                                poids = Transitions_with_action[transitions]['weights']
                                poids_total = np.sum(poids)
                                poids = poids/poids_total
                                poids = np.cumsum(poids)
                                choix = random.random()
                                for j in range(len(poids)):
                                    if choix <= poids[j]:
                                        reward_total += self.states[current_state].reward # ----------
                                        current_state = Transitions_with_action[transitions]['targets'][j]
                                        break
                                    
                    self.afficher(etat = current_state, file_name='image'+str(i+1))
                    images.append(imageio.imread('image'+str(i+1)+'.png'))
        imageio.mimsave(file_name+'.gif', images,fps=3)
        for i in range(n_iter+1):
            os.remove('image'+str(i)+'.png')
        for i in range(n_iter+1):
                os.remove('image'+str(i))

        print(f"Reward total : {reward_total}")
        
        return current_state

    def eventually_check_DTMC(self, S1):
        """ Model Checking du eventually pour une chaine de Markov sans action.
        
        Args
        ----
        S1 : list
            Liste des états qu'on cherche à atteindre
        """
        

        S_int = [x for x in self.liste_states if x not in S1] # on crée la liste des étates S_? ("int" pour interrogation)
        S0 = [] # on crée la liste des états S_0 qu'on remplira par récurrence
        S_int_2 = S_int.copy()

        for state in S_int : # Algorithme de recherche en profondeur qu'on applique à chaque état de S_int
            pile = []
            pile.append(state)
            etats_marques = [state]
            while pile != []:
                etat = pile[-1]
                etat_actif = self.states[etat]
                if etat in S1 :
                    break
                etats_faisables = [etat for etat in etat_actif.transitions_without_action["targets"] if etat not in etats_marques]
                if etats_faisables != [] :
                    pile.append(etats_faisables[0])
                    etats_marques.append(etats_faisables[0])
                else:
                    pile.pop()
            if pile == []:
                S_int_2.remove(state)
                S0.append(state) # on ajoute l'état à S0 si on ne peut pas atteindre un état de S1

        print(f"Les états S_0 sont : {S0}")
        print(f"Les états S_? sont : {S_int_2}")
        print(f"Les états S_1 sont : {S1}")

        A = np.zeros((len(S_int_2),len(S_int_2))) # Création de la matrice A
        for i in range(len(S_int_2)):
            etat_actif = self.states[S_int_2[i]]
            for j in range(len(S_int_2)):
                if S_int_2[j] in etat_actif.transitions_without_action["targets"]:
                    index = etat_actif.transitions_without_action["targets"].index(S_int_2[j])
                    A[i,j] = etat_actif.transitions_without_action["weights"][index]/(np.sum(etat_actif.transitions_without_action["weights"])) # on remplit la matrice A avec les bons poids

        b = np.zeros((len(S_int_2),1)) # Création du vecteur b
        for i in range(len(S_int_2)):
            etat_actif = self.states[S_int_2[i]]
            for transitions in etat_actif.transitions_without_action["targets"]:
                if transitions in S1:
                    index = etat_actif.transitions_without_action["targets"].index(transitions)
                    b[i,0] += etat_actif.transitions_without_action["weights"][index]/(np.sum(etat_actif.transitions_without_action["weights"])) # on remplit le vecteur b avec les bons poids

        # On résout (Id-A)^-1 * b
        Id = np.identity(len(S_int_2))
        y = np.dot(np.linalg.inv(Id - A),b)

        print(f"La probabilité de respecter la propriété à partir de l'état initial {self.liste_states[0]} est : {y[0,0]}")
        print(" vecteur y total :")
        print(y)

    def smc_quantitatif(self):
        print("Quel sont les états cibles ? Veuillez les rentrer un par un, et taper 'fin' quand vous avez fini")
        time.sleep(0.5)
        etat = input()
        etats_cibles = []
        while etat != "fin":
            if etat not in self.liste_states:
                print("L'état", etat, "n'existe pas")
                etat = input()
            else :
                etats_cibles.append(etat)
                etat = input()
        print("\nPrécision souhaitée :")
        time.sleep(0.5)
        precision = float(input())
        print("\nErreur souhaitée :")
        time.sleep(0.5)
        erreur = float(input())
        print("\nLongueur des chaines voulues :")
        time.sleep(0.5)
        longueur = int(input())

        # Calcul de N (nombre d'itérations)
        N =int((np.log(2)-np.log(erreur))/(2*precision)**2)+1

        print("\n----------------------------------")
        print("Récapitulatif des paramètres :")
        print("N = ", N, "\n")
        print("Precision (epsilon) = ", precision, "\n")
        print("Erreur (delta) = ", erreur, "\n")
        print("Longueur des chaines = ", longueur, "\n")
        print("----------------------------------")
        print("Calcul en cours.. \n")
        # On lance un certain nombre d'itération pour chaque état cible
        res_prob = [] # vecteur des probabilités d'atteindre les étates
        for i in range(len(etats_cibles)): 
            lancers = [markov.parcours_SMC(self, longueur, etats_cibles[i]) for _ in range(N)]
            res_prob.append(round(sum([1 for x in lancers if x==etats_cibles[i]])/N,5))
        print(f"Les probabilité d'atteindre chacun des états cibles {etats_cibles} sont : {res_prob}")

        return



    def __repr__(self):
        return (f"States : {self.states} \n Actions : {self.actions}")

### Mettre random directement dans parcours (le choix sera fait par un input)

In [14]:
M = markov(fichier_mdp="chaines/chaine_1.mdp")
# M.eventually_check_DTMC(["S3"])
# M.simulation_random(10, mode_adv = "random")


States: ['S0', 'S1', 'S2', 'S3', 'S4']
Actions: ['None']
Transition from S0 with no action and targets ['S1', 'S2'] with weights [5, 5]
Transition from S1 with no action and targets ['S0', 'S3'] with weights [5, 5]
Transition from S3 with no action and targets ['S3'] with weights [10]
Transition from S2 with no action and targets ['S4'] with weights [10]
Transition from S4 with no action and targets ['S1'] with weights [10]
{'States': ['S0', 'S1', 'S2', 'S3', 'S4'], 'Actions': ['None'], 'Transitions_with_action': {}, 'Transitions_without_action': {0: {'from': 'S0', 'targets': ['S1', 'S2'], 'weights': [5, 5]}, 1: {'from': 'S1', 'targets': ['S0', 'S3'], 'weights': [5, 5]}, 2: {'from': 'S3', 'targets': ['S3'], 'weights': [10]}, 3: {'from': 'S2', 'targets': ['S4'], 'weights': [10]}, 4: {'from': 'S4', 'targets': ['S1'], 'weights': [10]}}}
--------------------------------------------------


 Souhaitez-vous que les problèmes détéctés prochainement soient résolus automatiquement si cela est p